In [1]:
import os
import csv
from datetime import timedelta
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
"""
    1. Participants pulled from PRE-Flourish to Flourish, include gender, age, and BMI.
    2. Participants eligible for Flourish not pulled from PRE-Flourish
"""

'\n    1. Participants pulled from PRE-Flourish to Flourish, include gender, age, and BMI.\n    2. Participants eligible for Flourish not pulled from PRE-Flourish\n'

In [15]:
child_idxs = CaregiverChildConsent.objects.filter(
    study_child_identifier__icontains='P').values_list('subject_identifier', flat=True)
consents = []
for idx in child_idxs:
    latest = CaregiverChildConsent.objects.filter(subject_identifier=idx).latest('consent_datetime')
    # Check if BCPP before adding to consents list
    screening_idx = latest.subject_consent.screening_identifier
    dataset = MaternalDataset.objects.filter(screening_identifier=screening_idx).latest('created')
    if dataset.protocol == 'BCPP':
        cohort_obj = Cohort.objects.filter(subject_identifier=idx).latest('assign_datetime')
        consents.append({'subject_identifier': idx,
                         'study_child_identifier': latest.study_child_identifier,
                         'age': round(cohort_obj.child_age, 2),
                         'gender': cohort_obj.child_gender,
                         'bmi': round(cohort_obj.child_bmi, 2)})

In [18]:
keys = consents[0].keys()
with open('pre_flourish_to_flourish_pids.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(consents)

In [21]:
from pre_flourish.helper_classes.utils import is_flourish_eligible
from edc_constants.constants import YES

pf_child_idxs = PreFlourishCaregiverChildConsent.objects.values_list('subject_identifier', flat=True)
pf_consents = []

for idx in pf_child_idxs:
    flourish_enrol = CaregiverChildConsent.objects.filter(study_child_identifier=idx).exists()
    if is_flourish_eligible(idx) and not flourish_enrol:
        contact = PreFlourishContact.objects.filter(subject_identifier=idx,
                                                    contact_success=YES,
                                                    appt_scheduled=YES,
                                                    appt_date__isnull=False).last()
        scheduled_dt = getattr(contact, 'appt_date', '')
        pf_consents.append({'subject_identifier': idx,
                            'fl_enrol_scheduled': 'Yes' if scheduled_dt else 'No',
                            'scheduled_dt': scheduled_dt})

In [23]:
keys = pf_consents[0].keys()
with open('pf_flourish_eligible_pids.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(pf_consents)